In [51]:
import pandas as pd, numpy as np
import os
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score

In [52]:
# x_train
training_data = pd.read_csv('../data/original_data/noExclusion_train_data.csv', header = None)
# y_train
training_labels = pd.read_csv('../data/original_data/noExclusion_train_label.csv', header = None)
# x_test
testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
# y_test
testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)


In [30]:
# make CART classifier
clf_cart = tree.DecisionTreeClassifier(criterion="gini", random_state=1)
clf_cart = clf_cart.fit(training_data, training_labels)

# make predictions with CART model
accuracy = clf_cart.score(testing_data, testing_labels)
print(accuracy)

0.7762237762237763


In [55]:
clf_cart = tree.DecisionTreeClassifier(criterion="gini", random_state=1)
# find optimal parameter values for CART
params = {
    # 'max_depth': [None, 5, 10, 15, 20, 25, 30, 35, 40] # control overfitting,
    'max_features': [None, 'sqrt', 'log2']
}

grid_search = GridSearchCV(clf_cart, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, training_labels)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"best_params: {best_params} \nbest_score: {best_score}")


best_params: {'max_features': 'log2'} 
best_score: 0.806019358741682
